# Redshift Cluster using the AWS Python SDK

In [ ]:
import pandas as pd
import boto3
import json

### Step 0: Write IAM access key and secret in config file
Edit the file `dwh.cfg` in the same folder as this notebook and save the access key and secret against the following variables:
```bash
KEY= <YOUR_AWS_KEY>
SECRET= <YOUR_AWS_SECRET>
```

### Step 1: Load Parameters from config file
Assumption: IAM role for Redshift cluster already exists. Otherwise, create it with code below.

In [ ]:
import configparser
config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))

KEY                    = config.get('IAM USER','KEY')
SECRET                 = config.get('IAM USER','SECRET')

DWH_CLUSTER_TYPE       = config.get("CREATE CLUSTER","DWH_CLUSTER_TYPE")
DWH_NUM_NODES          = config.get("CREATE CLUSTER","DWH_NUM_NODES")
DWH_NODE_TYPE          = config.get("CREATE CLUSTER","DWH_NODE_TYPE")
DWH_IAM_ROLE_NAME      = config.get("CREATE CLUSTER", "DWH_IAM_ROLE_NAME")
DWH_IAM_ROLE_ARN       = config.get("CREATE CLUSTER", "DWH_IAM_ROLE_ARN")
DWH_VPC_SECURITY_GROUP_IDS  = config.get("CREATE CLUSTER", "DWH_VPC_SECURITY_GROUP_IDS")
DWH_CLUSTER_IDENTIFIER = config.get("CREATE CLUSTER","DWH_CLUSTER_IDENTIFIER")

DWH_DB                 = config.get("CLUSTER","DB_NAME")
DWH_DB_USER            = config.get("CLUSTER","DB_USER")
DWH_DB_PASSWORD        = config.get("CLUSTER","DB_PASSWORD")
DWH_PORT               = config.get("CLUSTER","DB_PORT")


(DWH_DB_USER, DWH_DB_PASSWORD, DWH_DB)

pd.DataFrame({"Param":
                  ["DWH_CLUSTER_TYPE", "DWH_NUM_NODES", "DWH_NODE_TYPE", "DWH_CLUSTER_IDENTIFIER", "DWH_DB", "DWH_DB_USER", "DWH_DB_PASSWORD", "DWH_PORT", "DWH_IAM_ROLE_NAME", "DWH_IAM_ROLE_ARN"],
              "Value":
                  [DWH_CLUSTER_TYPE, DWH_NUM_NODES, DWH_NODE_TYPE, DWH_CLUSTER_IDENTIFIER, DWH_DB, DWH_DB_USER, DWH_DB_PASSWORD, DWH_PORT, DWH_IAM_ROLE_NAME, DWH_IAM_ROLE_ARN]
             })

## # Step 2: Create clients for IAM, EC2, S3 and Redshift
**Note**: Creating resources in the **us-east-1** region. Choose the same region in AWS web console to the see these resources.

In [ ]:
ec2 = boto3.resource('ec2',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                    )

s3 = boto3.resource('s3',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                   )

iam = boto3.client('iam',aws_access_key_id=KEY,
                     aws_secret_access_key=SECRET,
                     region_name='us-west-2'
                  )

redshift = boto3.client('redshift',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                       )

### Step 3: Create Redshift Cluster
Create a [RedShift Cluster](https://console.aws.amazon.com/redshiftv2/home)
For complete arguments to `create_cluster`, see [docs](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/redshift.html#Redshift.Client.create_cluster)

In [ ]:
try:
    response = redshift.create_cluster(        
        #HW
        ClusterType=DWH_CLUSTER_TYPE,
        NodeType=DWH_NODE_TYPE,
        NumberOfNodes=int(DWH_NUM_NODES),

        #Identifiers & Credentials
        DBName=DWH_DB,
        ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,
        MasterUsername=DWH_DB_USER,
        MasterUserPassword=DWH_DB_PASSWORD,

        #Security Groups
        # VpcSecurityGroupIds=[DWH_VPC_SECURITY_GROUP_IDS], # this one is in us-east-1
        # https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/redshift.html#Redshift.Client.create_cluster

        #Roles (for s3 access)
        IamRoles=[DWH_IAM_ROLE_ARN]
    )
except Exception as e:
    print(e)

### Step 3.1 Describe the cluster to see its status
Run this block several times until the cluster status becomes `Available`

In [ ]:
def prettyRedshiftProps(props):
    pd.set_option('display.max_colwidth', 0)
    keysToShow = ["ClusterIdentifier", "NodeType", "ClusterStatus", "MasterUsername", "DBName", "Endpoint", "NumberOfNodes", 'VpcId', 'VpcSecurityGroups', 'IamRoles']
    x = [(k, v) for k,v in props.items() if k in keysToShow]
    return pd.DataFrame(data=x, columns=["Key", "Value"])

myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

### Step 3.2: Take note of endpoint and role ARN of Redshift Cluster
Do not run this before the cluster status becomes "Available".

DWH_ENDPOINT is needed to access Redshift cluster
ROLE_ARN is needed in statement to load data into Redshift from S3

In [ ]:
DWH_ENDPOINT = myClusterProps['Endpoint']['Address']
DWH_ROLE_ARN = myClusterProps['IamRoles'][0]['IamRoleArn']
print("DWH_ENDPOINT :: ", DWH_ENDPOINT)
print("DWH_ROLE_ARN :: ", DWH_ROLE_ARN)

### Step 4: Make sure you can connect to the cluster

In [ ]:
%load_ext sql

In [ ]:
conn_string="postgresql://{}:{}@{}:{}/{}".format(DWH_DB_USER, DWH_DB_PASSWORD, DWH_ENDPOINT, DWH_PORT,DWH_DB)
print(conn_string)
%sql $conn_string

### Step 5: Delete Redshift Cluster

In [ ]:
#### CAREFUL!!
redshift.delete_cluster( ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,  SkipFinalClusterSnapshot=True)
#### CAREFUL!!

#### Run this block several times until the cluster really deleted

In [ ]:
myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

***

#### Optional: Take a look at S3 bucket data

In [ ]:
sampleDbBucket =  s3.Bucket("udacity-dend")
for obj in sampleDbBucket.objects.filter(Prefix="song-data/M"):
    print(obj)

#s3.Bucket("udacity-dend").download_file('log_json_path.json', 'log_json_path.json')

#data = sampleDbBucket.get_object(Bucket='mudacity-dend', Key='log_json_path.json')
#contents = data['Body'].read()
#print(contents)

# for obj in sampleDbBucket.objects.all():
#     print(obj)

### Optional: Create IAM role for Redshift Cluster
Create an IAM Role that makes Redshift able to access S3 bucket (ReadOnly)

In [ ]:
from botocore.exceptions import ClientError

#1.1 Create the role,
try:
    print("1.1 Creating a new IAM Role")
    dwhRole = iam.create_role(
        Path='/',
        RoleName=DWH_IAM_ROLE_NAME,
        Description = "Allows Redshift clusters to call AWS services on your behalf.",
        AssumeRolePolicyDocument=json.dumps(
            {'Statement': [{'Action': 'sts:AssumeRole',
               'Effect': 'Allow',
               'Principal': {'Service': 'redshift.amazonaws.com'}}],
             'Version': '2012-10-17'})
    )
except Exception as e:
    print(e)


print("1.2 Attaching Policy")

iam.attach_role_policy(RoleName=DWH_IAM_ROLE_NAME,
                       PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
                      )['ResponseMetadata']['HTTPStatusCode']

print("1.3 Get the IAM role ARN")
roleArn = iam.get_role(RoleName=DWH_IAM_ROLE_NAME)['Role']['Arn']

print(roleArn)

### Optional: Open an incoming TCP port to access the cluster endpoint as access rule in security group

In [ ]:
try:
    # get VPC our cluster is running on
    vpc = ec2.Vpc(id=myClusterProps['VpcId'])
    # get list of security groups in VPC (or first entry)
    defaultSg = list(vpc.security_groups.all())[0]
    print(defaultSg)
    # Add the specified inbound (ingress) rules to a security group
    defaultSg.authorize_ingress(
        GroupName=defaultSg.group_name,
        CidrIp='0.0.0.0/0',
        IpProtocol='TCP',
        FromPort=int(DWH_PORT),
        ToPort=int(DWH_PORT)
    )
except Exception as e:
    print(e)

### Optional: Delete IAM ROLE for Redshift Cluster

In [ ]:
#-- Uncomment & run to delete the created resources
# iam.detach_role_policy(RoleName=DWH_IAM_ROLE_NAME, PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess")
# iam.delete_role(RoleName=DWH_IAM_ROLE_NAME)